In [28]:
%pwd

u'C:\\Users\\Joey-YongChang.Huang@cn.abb.com\\kamidox\\work\\stock-data-raw'

In [29]:
import pandas as pd
import numpy as np

In [30]:
d = pd.read_csv('2008/SH600300.csv', names=['date','time','opening_price', 'ceiling_price', 'floor_price', 'closing_price', 'volume', 'amount'], header=None)

In [31]:
d.head()

,date,time,opening_price,ceiling_price,floor_price,closing_price,volume,amount
0,2008/01/02,09:35,9.89,9.90,9.76,9.77,3223.00,3174676
1,2008/01/02,09:40,9.77,9.84,9.75,9.79,2037.50,1992715
2,2008/01/02,09:45,9.78,9.82,9.78,9.79,1227.99,1203187
3,2008/01/02,09:50,9.80,9.85,9.79,9.85,1137.00,1115308
4,2008/01/02,09:55,9.84,9.95,9.84,9.94,1422.97,1405492


In [32]:
g = d.groupby('date')

In [33]:
day = g.agg({'opening_price': 'first', 'ceiling_price': 'max', 'floor_price': 'min', 'closing_price': 'last', 'volume': 'sum', 'amount': 'sum'})

In [34]:
day.head()

,floor_price,opening_price,ceiling_price,volume,amount,closing_price
date,,,,,,
2008/01/02,9.75,9.89,10.31,87545.17,88759296,10.22
2008/01/03,10.22,10.31,10.88,151147.35,159856480,10.58
2008/01/04,10.28,10.58,10.76,81476.55,85745624,10.58
2008/01/07,10.45,10.48,11.09,130950.71,141612400,10.98
2008/01/08,10.30,10.95,11.00,116279.70,124193984,10.42


In [35]:
day.to_csv('SH600300_day.csv', mode='a')

In [39]:
day.columns.sort_values(ascending=True)

Index([u'amount', u'ceiling_price', u'closing_price', u'floor_price',
       u'opening_price', u'volume'],
      dtype='object')

In [40]:
day.head()

,floor_price,opening_price,ceiling_price,volume,amount,closing_price
date,,,,,,
2008/01/02,9.75,9.89,10.31,87545.17,88759296,10.22
2008/01/03,10.22,10.31,10.88,151147.35,159856480,10.58
2008/01/04,10.28,10.58,10.76,81476.55,85745624,10.58
2008/01/07,10.45,10.48,11.09,130950.71,141612400,10.98
2008/01/08,10.30,10.95,11.00,116279.70,124193984,10.42
